In [ ]:
!pip install torchmetrics

     |████████████████████████████████| 397 kB 4.1 MB/s 


In [ ]:
!pip install ipdb

     |████████████████████████████████| 793 kB 4.1 MB/s 
     |████████████████████████████████| 380 kB 44.7 MB/s 
  Created wheel for ipdb: filename=ipdb-0.13.9-py3-none-any.whl size=11648 sha256=a2d9fb281ca050f1ae5d842e216ccc7a3ee86f30317e376dac81284c2e73a03a
  Stored in directory: /root/.cache/pip/wheels/65/cd/cc/aaf92acae337a28fdd2aa4d632196a59745c8c39f76eaeed01
Successfully built ipdb
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Attempting uninstall: ipython
    Found existing installation: ipython 5.5.0
    Uninstalling ipython-5.5.0:
      Successfully uninstalled ipython-5.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 

In [ ]:
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
from string import punctuation
from collections import Counter
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence
import torch.optim as optim
from torchmetrics import F1
from torchmetrics.functional import f1, recall
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Скачивание и подготовка данных

In [ ]:
train_data = pd.read_csv('/content/gdrive/MyDrive/data/train.csv')
train_data = train_data[train_data['Class'] != 'Human']

In [ ]:
val_data =  pd.read_csv('/content/gdrive/MyDrive/data/val.csv')
val_data = val_data[val_data['Class'] != 'Human']

In [ ]:
vocab = Counter()
for text in tqdm(train_data.Text):
    for symbol in text:
        if symbol is not "\n":
            vocab.update(list(symbol))
print('всего уникальных символов:', len(vocab))

  0%|          | 0/77916 [00:00<?, ?it/s]

всего уникальных символов: 832


In [ ]:
#создаем словарь с индексами symbol2id, для спецсимвола паддинга дефолтный индекс - 0
symbol2id = {'PAD':0}

for symbol in tqdm(vocab):
    symbol2id[symbol] = len(symbol2id)

  0%|          | 0/832 [00:00<?, ?it/s]

In [ ]:
#обратный словарь для того, чтобы раскодировать последовательность
id2symbol = {i:symbol for symbol, i in symbol2id.items()}

In [ ]:
label_encoder = LabelEncoder()
train_data['label'] = label_encoder.fit_transform(train_data['Class'])
#train_data['label'] = np.where((train_data.label > 1), 1, train_data.label)

In [ ]:
val_data['label'] = label_encoder.transform(val_data['Class'])
#val_data['label'] = np.where((val_data.label > 1), 1, val_data.label)

In [ ]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
DEVICE

device(type='cuda')

## Dataset & DataLoader

In [ ]:
class CharacterDataset(Dataset):

    def __init__(self, dataset, symbol2id, DEVICE):
        self.dataset = dataset['Text'].values
        self.symbol2id = symbol2id
        self.length = dataset.shape[0]
        self.target = dataset['label'].values
        self.device = DEVICE

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        texts = self.dataset[index] 
        symbols = list(self.dataset[index])
        ids = torch.LongTensor([self.symbol2id[symbol] for symbol in symbols if symbol in self.symbol2id])
        y = self.target[index]
        return ids, y, texts

    def collate_fn(self, batch): #этот метод можно реализовывать и отдельно,
    # он понадобится для DataLoader во время итерации по батчам
      ids, y, texts = list(zip(*batch))
      padded_ids = pad_sequence(ids, batch_first=True).to(self.device)
      y = torch.LongTensor(y).to(self.device)
      return padded_ids, y, texts

In [ ]:
train_dataset = CharacterDataset(train_data, symbol2id, DEVICE)
train_sampler = RandomSampler(train_dataset)
train_iterator = DataLoader(train_dataset, collate_fn = train_dataset.collate_fn, sampler=train_sampler, batch_size=64)

In [ ]:
val_dataset = CharacterDataset(val_data, symbol2id, DEVICE)
val_sampler = SequentialSampler(val_dataset)
val_iterator = DataLoader(val_dataset, collate_fn = val_dataset.collate_fn, sampler=val_sampler, batch_size=64)

## CNN model

In [ ]:
class CNN(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.bigrams = nn.Conv1d(in_channels=embedding_dim, out_channels=100, kernel_size=2, padding='same')
        self.trigrams = nn.Conv1d(in_channels=embedding_dim, out_channels=80, kernel_size=3, padding='same')
        self.pooling = nn.MaxPool1d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()
        self.hidden = nn.Linear(in_features=180, out_features=13)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, word):
        #batch_size x seq_len
        embedded = self.embedding(word)
        #batch_size x seq_len x embedding_dim
        embedded = embedded.transpose(1,2)
        #batch_size x embedding_dim x seq_len
        feature_map_bigrams = self.dropout(self.pooling(self.relu(self.bigrams(embedded))))
        #batch_size x filter_count2 x seq_len* 
        feature_map_trigrams = self.dropout(self.pooling(self.relu(self.trigrams(embedded))))
        #batch_size x filter_count3 x seq_len*

        pooling1 = feature_map_bigrams.max(2)[0] 
        # batch_size x filter_count2
        pooling2 = feature_map_trigrams.max(2)[0]
        # batch_size x filter_count3
        concat = torch.cat((pooling1, pooling2), 1)
        # batch _size x (filter_count2 + filter_count3)
        logits = self.hidden(concat) 
        #logits = self.out(logits)      
        return logits

In [ ]:
loss = nn.CrossEntropyLoss()

## training loop, логика обучения и валидации

In [ ]:
def train(model, iterator, optimizer, criterion, df):
    epoch_loss = 0 # для подсчета среднего лосса на всех батчах
    model.train()  # ставим модель в обучение, явно указываем, что сейчас надо будет хранить градиенты у всех весов
    predictions = []
    acc = 0
    
    for i, (symbols, ys, texts) in enumerate(iterator): #итерируемся по батчам
        optimizer.zero_grad()  #обнуляем градиенты
        preds = model(symbols)
        loss = criterion(preds, ys) #считаем значение функции потерь
        loss.backward() #считаем градиенты  
        preds = preds.cpu().detach().numpy()  #прогоняем данные через модель
        predictions.extend(np.round(preds))
        optimizer.step() #обновляем веса 
        epoch_loss += loss.item() #сохраняем значение функции потерь
        if not (i + 1) % int(len(iterator)/5):
            print(f'Train loss: {epoch_loss/i}')
        acc += accuracy_score(ys.cpu().detach().numpy(), preds.argmax(axis=1))
    print(f'Train accuracy: {acc/len(iterator)}')      

In [ ]:
def evaluate(model, iterator, criterion, df):
    epoch_loss = 0
    accuracy = []
    predictions = []
    model.eval() 
    text = []
    labels = []
    with torch.no_grad():
        for i, (symbols, ys, texts) in enumerate(iterator):   
            preds = model(symbols)  # делаем предсказания на тесте
            loss = criterion(preds, ys)   # считаем значения функции ошибки для статистики  
            epoch_loss += loss.item()
            preds = preds.cpu().detach().numpy()
            predictions.extend(preds.argmax(axis=1))
            labels.extend(ys.cpu().detach().numpy())
            text.extend(list(texts))
            accuracy.append(accuracy_score(ys.cpu().detach().numpy(), preds.argmax(axis=1)))
        print(f'Val accuracy: {sum(accuracy)/ len(iterator)}') # возвращаем среднее значение по всей выборке
    return predictions, labels, text

## инициализируем модель, задаем оптимизатор и функцию потерь

In [ ]:
model = CNN(len(symbol2id), 100)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss() 

# веса модели и значения лосса храним там же, где и все остальные тензоры
model = model.to(DEVICE)
criterion = criterion.to(DEVICE)

## запуск обучения

In [ ]:
losses = []
losses_eval = []
f1s = []
f1s_eval = []

for i in tqdm(range(10)):
    print(f'\nstarting Epoch {i}')
    print('Training...')
    train(model, train_iterator, optimizer, criterion, train_data)
    print('\nEvaluating on test...')
    evaluate(model, val_iterator, criterion, val_data)

  0%|          | 0/10 [00:00<?, ?it/s]


starting Epoch 0
Training...


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/conv.py:298: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  ../aten/src/ATen/native/Convolution.cpp:647.)
  self.padding, self.dilation, self.groups)


Train loss: 2.1756445400971027
Train loss: 2.0925699646939937
Train loss: 2.0435246859605494
Train loss: 2.0073273235933913
Train loss: 1.9777724317703058
Train accuracy: 0.26354130014074595

Evaluating on test...
Val accuracy: 0.31501654212240127

starting Epoch 1
Training...
Train loss: 1.85339451328782
Train loss: 1.8353490195323512
Train loss: 1.8218555707525421
Train loss: 1.8119500680269345
Train loss: 1.8024028938331165
Train accuracy: 0.3158353858784893

Evaluating on test...
Val accuracy: 0.31889115596560075

starting Epoch 2
Training...
Train loss: 1.7420542456886985
Train loss: 1.7325440082353416
Train loss: 1.733520010506714
Train loss: 1.7303206747518631
Train loss: 1.7269502549234292
Train accuracy: 0.3402038617171006

Evaluating on test...
Val accuracy: 0.32903168573098435

starting Epoch 3
Training...
Train loss: 1.6897495699322913
Train loss: 1.6910293987116862
Train loss: 1.690046903851268
Train loss: 1.6859389299221805
Train loss: 1.6837181587392063
Train accuracy: 0

In [ ]:
predictions, labels, texts = evaluate(model, val_iterator, criterion, val_data)
df = pd.DataFrame({'texts':texts, 'predictions':label_encoder.inverse_transform(predictions),
                   'labels':label_encoder.inverse_transform(labels)})

Val accuracy: 0.3743033522584954


In [ ]:
df.to_csv('CNN_character.csv')

## работа с моделью

In [ ]:
torch.save(model.state_dict(), 'model.pth') 